**  KAGGLE AUTHENTICATION THROUGH UPLOADING API FILE**

In [0]:
import zipfile
from google.colab import auth
auth.authenticate_user()

In [0]:
import os
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ferdi33","key":"98cb2f31922e43ead612a3740a2b75f7"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [0]:
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json

total 4
-rw------- 1 root root 63 May  8 07:08 kaggle.json
{"username":"ferdi33","key":"98cb2f31922e43ead612a3740a2b75f7"}

In [0]:
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 3.1MB/s 
     |████████████████████████████████| 5.3MB 8.3MB/s 
     |████████████████████████████████| 51kB 21.0MB/s 
     |████████████████████████████████| 102kB 31.0MB/s 


**DOWNLOADING AND UNZIPPING FILES**

In [0]:
!kaggle competitions download -c histopathologic-cancer-detection

  0% 0.00/1.33M [00:00<?, ?B/s]
100% 1.33M/1.33M [00:00<00:00, 43.7MB/s]
  0% 0.00/5.10M [00:00<?, ?B/s]
100% 5.10M/5.10M [00:00<00:00, 46.8MB/s]
100% 1.30G/1.30G [00:12<00:00, 108MB/s]
100% 1.30G/1.30G [00:12<00:00, 109MB/s]
100% 4.98G/4.98G [00:43<00:00, 150MB/s]
100% 4.98G/4.98G [00:43<00:00, 122MB/s]


In [0]:

zipf = zipfile.ZipFile('../content/train.zip')
zipf.extractall('../content/train')

In [0]:
zipf = zipfile.ZipFile('../content/test.zip')
zipf.extractall('../content/test/test')

In [0]:
! rm -r ../content/train_labels.csv
zipf = zipfile.ZipFile('../content/train_labels.csv.zip')
zipf.extractall()

In [0]:
# Añadir la extensión .tiff en el csv con los labels, para que el flow_from_dataframe pueda leer las imagenes.
import pandas as pd
df = pd.read_csv('../content/train_labels.csv')

df['label'] = df['label'].astype(str)
df['id'] = df['id'].apply(lambda x: x+'.tif')

df.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835.tif,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif,1
2,755db6279dae599ebb4d39a9123cce439965282d.tif,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif,0
4,068aba587a4950175d04c680d38943fd488d6a9d.tif,0


In [0]:
# Split en validation/train
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, test_size=0.2)

In [0]:
import keras
from keras_preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
datagen=ImageDataGenerator(rescale=1./255,
                          rotation_range=45,
                          horizontal_flip=True,
                          vertical_flip=True,
                          )

train_generator=datagen.flow_from_dataframe(dataframe=train_df, 
                                            directory="../content/train", 
                                            x_col="id", y_col="label", 
                                            class_mode="categorical", 
                                            target_size=(32,32), 
                                            batch_size=256)

validation_generator=datagen.flow_from_dataframe(dataframe=valid_df, 
                                                 directory="../content/train", 
                                                 x_col="id", y_col="label", 
                                                 class_mode="categorical", 
                                                 target_size=(32,32), 
                                                 batch_size=256)

Found 176020 images belonging to 2 classes.
Found 44005 images belonging to 2 classes.


**NN MODEL**

In [59]:
from keras import models
from keras import layers


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(12, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_5 (Flatten)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 1024)              4096      
__________

In [42]:
n_epochs = 5
batch_size=64

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = n_epochs)

Epoch 1/5
2750/2750 [==============================] - 1831s 666ms/step - loss: 0.4017 - acc: 0.8203 - val_loss: 0.4567 - val_acc: 0.7667
Epoch 2/5
2750/2750 [==============================] - 1864s 678ms/step - loss: 0.3450 - acc: 0.8503 - val_loss: 0.5920 - val_acc: 0.7201
Epoch 3/5
2750/2750 [==============================] - 1814s 660ms/step - loss: 0.3207 - acc: 0.8625 - val_loss: 1.0979 - val_acc: 0.6848
Epoch 4/5
2750/2750 [==============================] - 1889s 687ms/step - loss: 0.3059 - acc: 0.8694 - val_loss: 0.7486 - val_acc: 0.7474
Epoch 5/5
2750/2750 [==============================] - 1883s 685ms/step - loss: 0.2941 - acc: 0.8753 - val_loss: 0.8263 - val_acc: 0.6982


In [0]:
import matplotlib.pyplot as plt
plt.plot(model.history.history['loss'], label='Train Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

**GENERATE THE PREDICTIONS**





In [44]:
test_dir = '../content/test/test'
test_datagen = ImageDataGenerator(rescale=1./255)


print(os.getcwd()+os.path.sep+test_dir)
test_generator = test_datagen.flow_from_directory(
        '../content/test',
        target_size=(32, 32),
        color_mode="rgb",
        batch_size=2,
        shuffle=False)
filenames = test_generator.filenames
nb_samples = len(filenames)

/content/../content/test/test
Found 57458 images belonging to 1 classes.


In [45]:
predictions = model.predict_generator(test_generator,steps = nb_samples//2, verbose=1)

28729/28729 [==============================] - 2909s 101ms/step


In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [51]:
predictions[:10]

array([[0.01888039, 0.98111963],
       [0.4892215 , 0.5107785 ],
       [0.25815332, 0.7418467 ],
       [0.9434368 , 0.05656321],
       [0.6696484 , 0.33035162],
       [0.43569696, 0.56430304],
       [0.9520116 , 0.04798849],
       [0.6249113 , 0.37508875],
       [0.9930335 , 0.00696652],
       [0.85141355, 0.14858651]], dtype=float32)

In [0]:
import numpy as np
predicted_class_indices=np.argmax(predictions,axis=1)

In [53]:
predicted_class_indices[:10]

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 0])

In [0]:
predictions = [labels[k] for k in predicted_class_indices]

**UPLOAD THE PREDICTIONS TO KAGGLE**

In [55]:
filenames=test_generator.filenames
print(filenames[:10])

['test/00006537328c33e284c973d7b39d340809f7271b.tif', 'test/0000ec92553fda4ce39889f9226ace43cae3364e.tif', 'test/00024a6dee61f12f7856b0fc6be20bc7a48ba3d2.tif', 'test/000253dfaa0be9d0d100283b22284ab2f6b643f6.tif', 'test/000270442cc15af719583a8172c87cd2bd9c7746.tif', 'test/000309e669fa3b18fb0ed6a253a2850cce751a95.tif', 'test/000360e0d8358db520b5c7564ac70c5706a0beb0.tif', 'test/00040095a4a671280aeb66cb0c9231e6216633b5.tif', 'test/000698b7df308d75ec9559ef473a588c513a68aa.tif', 'test/0006e1af5670323331d09880924381d67d79eda0.tif']


In [56]:
# Remove .tiff extension and /test at the beginning
filenames = [x[:-4] for x in filenames]
filenames = [x[5:] for x in filenames]
print(filenames[:5])

['00006537328c33e284c973d7b39d340809f7271b', '0000ec92553fda4ce39889f9226ace43cae3364e', '00024a6dee61f12f7856b0fc6be20bc7a48ba3d2', '000253dfaa0be9d0d100283b22284ab2f6b643f6', '000270442cc15af719583a8172c87cd2bd9c7746']


In [0]:
results=pd.DataFrame({"id":filenames,
                      "label":predictions})
results.to_csv("results.csv",index=False)

In [58]:
import kaggle
!kaggle competitions submit histopathologic-cancer-detection -f "../content/results.csv" -m "Automatic Submission"

100% 2.36M/2.36M [00:00<00:00, 3.45MB/s]
Successfully submitted to Histopathologic Cancer Detection